

# Recipe #1:  Autosuggest



For simple autosuggest, we need to index the following data-shape:  

(**last_query_date**, **query**, **count**)

In [ ]:
import sys
!conda install --yes --prefix {sys.prefix} elasticsearch jsonlines jsonpath-ng python-slugify

import pandas
from extract import extract, parse_json
from autosuggest.transform import reshape, preload
from index import create_index, populate_index, query_index, index_mapping, query_template, peek



## Source data 


In [ ]:
source = extract('./data/real_estate_queries.tsv')

# peek
source.head(5)


##  Transform to fit the recipe


In [ ]:
reshaped = reshape(source)

# peek
reshaped.head(5)

E.g. 189 distinct users searched for "prudential real estate"

In [ ]:
load_ready = preload(reshaped)

# peek
[print(j) for j in load_ready[0:10]]



## Load the index


In [ ]:
create_index(index='autosuggest', mapping_file='autosuggest/index_definition.json')

index_mapping(index='autosuggest')

In [ ]:
response = populate_index(body=load_ready)


## Query for suggestions


In [ ]:
peek(filename='autosuggest/query_template.mustache')

In [ ]:
result = query_template(index='autosuggest', 
               template_file='autosuggest/query_template.mustache',
               params={ 'text': 'presc', 'size': 10 }
              )

suggestions = parse_json(result, jsonpath="hits.hits[*]._source.query")
print(*suggestions, sep='\n')